# Big Data Modelling and Analytics

Analysis on White House Visitors list

Dataset link - https://obamawhitehouse.archives.gov/sites/default/files/disclosures/whitehouse_waves-2016_12.csv_.zip

In [159]:
from pyspark.sql import SparkSession
spark = SparkSession\
      .builder\
      .appName("whitehouse")\
      .getOrCreate()

In [160]:
input_path = './whitehouse_waves-2016_12.csv'

In [161]:
df = spark.read.format("csv").option("header","true").option("inferSchema", "true").load(input_path)

In [162]:
neccesary_columns_df = df.select('NAMELAST','NAMEFIRST','NAMEMID','visitee_namelast','visitee_namefirst')

In [163]:
neccesary_columns_df.createOrReplaceTempView("Initialrecords")

In [164]:
lowercase_df = spark.sql("select lower(NAMELAST) as NAMELAST, lower(NAMEFIRST) as NAMEFIRST, lower(NAMEMID) as NAMEMID, lower(visitee_namelast) as visitee_namelast, lower(visitee_namefirst) as visitee_namefirst from Initialrecords ")

In [165]:
lowercase_df.show()

+---------------+---------+-------+----------------+-----------------+
|       NAMELAST|NAMEFIRST|NAMEMID|visitee_namelast|visitee_namefirst|
+---------------+---------+-------+----------------+-----------------+
|tajouribessassi|   hanene|   null|        pelofsky|             eric|
|        bageant|    laura|      j|     baskerville|           steven|
|       broemson|     earl|      h|     baskerville|           steven|
|    jackling jr|  william|      c|     baskerville|           steven|
|        mccrary|  richard|      l|     baskerville|           steven|
|        mulcahy|   joshua|      e|     baskerville|           steven|
|           ryan|   oliver|      j|     baskerville|           steven|
|       smith jr|  william|      t|     baskerville|           steven|
|         keeler|  douglas|      e|       goldstein|             jeff|
|          davis|   justin|      a|            drew|              maj|
|         glover|   vinson|      n|         lengyel|            jason|
|     

In [166]:
lowercase_df.count()

970504

### Filtering and Removing null and non aplha last name records

In [167]:
noNull_df = lowercase_df.filter((f.col('visitee_namelast')!='')&(f.col('NAMELAST')!=''))

In [168]:
noNull_df.count()

911249

In [169]:
noNull_df.createOrReplaceTempView("total_records")

In [170]:
valid_df = spark.sql("select * from total_records where visitee_namelast REGEXP '^[a-z]+$' and NAMELAST REGEXP '^[a-z]+$'")

In [171]:
valid_df.count()

897037

In [172]:
Number_of_records_before_filtering = lowercase_df.count()

In [173]:
Number_of_records_after_filtering = valid_df.count()

In [174]:
Number_of_records_before_filtering

970504

In [175]:
Number_of_records_after_filtering

897037

### Part 4 - The number of records dropped

In [176]:
Number_of_records_before_filtering - Number_of_records_after_filtering

73467

### Converting to a view

In [177]:
valid_df.createOrReplaceTempView("Filtered_records")

### Part 1 - The 10 most frequent visitors
(NAMELAST, NAMEFIRST, NAMEMID)
to the White House.

In [178]:
visitors = spark.sql("select NAMELAST,NAMEFIRST,NAMEMID, count(*) as total_visits FROM Filtered_records GROUP BY NAMELAST,NAMEFIRST,NAMEMID order by total_visits desc")

In [179]:
visitors.show(10)

+--------+---------+-------+------------+
|NAMELAST|NAMEFIRST|NAMEMID|total_visits|
+--------+---------+-------+------------+
|  thomas| benjamin|      l|         185|
|  berner|katherine|      k|         176|
|    haas|   jordan|      m|         152|
|   grant|  patrick|      c|         151|
| kidwell|   lauren|      k|         145|
|    haro|   steven|      m|         140|
|   garza|   steven|      a|         127|
|  strait|     elan|   null|         107|
|     lew| shoshana|      m|         102|
| zeitlin|   daniel|      l|          98|
+--------+---------+-------+------------+
only showing top 10 rows



### Part 2 - The 10 most frequently visited people
(visitee_namelast, visitee_namefirst)
in the White House.

In [180]:
visitees = spark.sql("select visitee_namelast,visitee_namefirst, count(*) as total_visits FROM Filtered_records GROUP BY visitee_namelast,visitee_namefirst order by total_visits desc")

In [181]:
visitees.show(10)

+----------------+-----------------+------------+
|visitee_namelast|visitee_namefirst|total_visits|
+----------------+-----------------+------------+
|          office|         visitors|      430721|
|           waves|   visitorsoffice|       44115|
|          bryant|             ruth|       13970|
|           oneil|           olivia|       13155|
|        thompson|            jared|       11605|
|          burton|           collin|        9672|
|           megan|          matthew|        7943|
|        mayerson|            asher|        6885|
|      dessources|          kalisha|        5285|
|           evans|            karen|        2908|
+----------------+-----------------+------------+
only showing top 10 rows



### Part 3 - The 10 most frequent visitor-visitee
combinations.

In [182]:
visitor_visitee = spark.sql("select  NAMELAST,NAMEFIRST,NAMEMID,visitee_namelast,visitee_namefirst, count(*) as total_visits FROM Filtered_records GROUP BY  NAMELAST,NAMEFIRST,NAMEMID,visitee_namelast,visitee_namefirst order by total_visits desc")

In [183]:
visitor_visitee.show(10)

+--------+---------+-------+----------------+-----------------+------------+
|NAMELAST|NAMEFIRST|NAMEMID|visitee_namelast|visitee_namefirst|total_visits|
+--------+---------+-------+----------------+-----------------+------------+
|    haas|   jordan|      m|        yudelson|             alex|          90|
|  thomas| benjamin|      l|        yudelson|             alex|          89|
|   grant|  patrick|      c|        yudelson|             alex|          88|
|  berner|katherine|      k|        yudelson|             alex|          82|
|   roche|  shannon|      e|        yudelson|             alex|          70|
|  urizar| jennifer|      a|         johnson|            katie|          68|
|  martin|  kathryn|   null|         lambrew|           jeanne|          56|
| kidwell|   lauren|      k|         abraham|         yohannes|          55|
|  berner|katherine|      k|         abraham|         yohannes|          54|
|angerman|elizabeth|   null|           mader|            david|          54|